<a href="https://colab.research.google.com/github/Lihini827/Diabetic-retinopathy-detection/blob/main/Aug_preprocessing(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from PIL import Image
from imgaug import augmenters as iaa
import numpy as np
import glob,os,cv2

import warnings
warnings.filterwarnings('ignore')

In [ ]:
path_parent = '/content/drive/MyDrive/DiabticRetinopathy/Preprocess/Ben/train/*'
path_parent = os.path.dirname(os.path.abspath(path_parent)) + '/'
dirs_parent = os.listdir(path_parent)

In [ ]:
print(dirs_parent)

['Mild']


In [ ]:
for item_parent in dirs_parent:
    print(item_parent)

    path = '/content/drive/MyDrive/DiabticRetinopathy/Preprocess/Ben/train/'+item_parent+'/*'
    path = os.path.dirname(os.path.abspath(path)) + '/'
    dirs = os.listdir(path)

Mild


In [ ]:
    for item in dirs:
        if (os.path.isfile(path+item)) & ('png' in item):
            im = Image.open(path+item)
            if im.mode !='RGB':
                im = im.convert('RGB')
            imResize = im.resize((224,224),Image.ANTIALIAS)
            imResize.save(path+item,'png',quality = 90)
    fps = glob.glob(path+'*')
    images = np.array(
        [cv2.cvtColor(cv2.imread(fp),cv2.COLOR_BGR2RGB) for fp in fps],
        dtype = np.uint8
    )

In [ ]:
    seq = iaa.Sequential([
    iaa.Fliplr(1.0),
    iaa.Flipud(1.0),
    iaa.Crop(percent = (0,0.1)),
    iaa.Sometimes(0.5,
        iaa.GaussianBlur(sigma=(0,0.5))
    ),
    iaa.ContrastNormalization((0.75,1.5)),#strengthen or weaken the contrast in each image
    iaa.AdditiveGaussianNoise(loc=0,scale=(0.0,0.05*255),per_channel=0.5),#add gaussian Noise
    iaa.Multiply((0.8,1.2),per_channel=0.2),#configure Lightness


    iaa.Affine(    #Affine Transform : Scale/Zoom them, translate/move them, rotate them and shear them)
        scale={"x":(0.8,1.2),"y":(0.8,1.2)},
        translate_percent={"x":(-0.2,0.2),"y":(-0.2,0.2)},
        rotate=(-45,45),
        shear=(-16,16))
    ],random_order=True)

In [ ]:
aug_times = 2


for times in range(aug_times):
  images_aug = seq(images=images)

  i=0
  for img in images_aug:
    cv2.imwrite(os.path.join( path,f'{times}'+'augmented_'+os.path.basename(fps[i])),cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
    i+=1